In [1]:
import dataclasses
import json
import logging
import os

import torch
from IPython.core.interactiveshell import InteractiveShell
from llama_head import CEL_only_forward
from peft import LoraConfig
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    LlamaTokenizer,
    Trainer,
    TrainerCallback,
    TrainingArguments,
)
from transformers.models.llama import LlamaConfig, LlamaForCausalLM
from transformers.trainer_callback import ProgressCallback
from transformers.trainer_pt_utils import _secs2timedelta
from trl import SFTTrainer

import unsloth.utils.data as data_utils
import unsloth.utils.memory as memory_utils
import unsloth.utils.testing as test_utils
from unsloth.kernels import fused_cel
from unsloth.kernels.fused_cel import patch_model as patch_model_fused_cel
from unsloth.models._utils import patch_tokenizer, prepare_model_for_kbit_training
from unsloth.models.llama import FastLlamaModel
from unsloth.utils.profiling import MetricsCallBack

logging.basicConfig(level=logging.WARNING)

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=False,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model_config = LlamaConfig.from_pretrained("./llama-10m.json")
model = AutoModelForCausalLM.from_pretrained(
    "./llama-10m", quantization_config=quant_config, torch_dtype=torch.bfloat16
)
# model = LlamaForCausalLM(model_config).to("cuda")

tokenizer = AutoTokenizer.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf", model_max_length=4096, padding_side="right"
)
model, tokenizer = patch_tokenizer(model, tokenizer)

max_seq_length = 256

training_args = TrainingArguments(
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    warmup_steps=5,
    max_steps=5,
    learning_rate=2e-4,
    fp16=not torch.cuda.is_bf16_supported(),
    bf16=torch.cuda.is_bf16_supported(),
    logging_steps=1,
    optim="adamw_8bit",
    weight_decay=0.01,
    lr_scheduler_type="linear",
    seed=3407,
    output_dir="outputs",
    overwrite_output_dir=True,
    # Metrics
    skip_memory_metrics=False,
    include_num_input_tokens_seen=True,
    include_tokens_per_second=True,
)

accepted_modules = frozenset(
    (
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ),
)

dataset = data_utils.get_alpaca(tokenizer)

peft_config = LoraConfig(
    target_modules=accepted_modules,
    lora_alpha=8,
    lora_dropout=0.0,
    bias="none",
    task_type="CAUSAL_LM",
)

WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.3.0+cu121 with CUDA 1201 (you have 2.4.0.dev20240507+cu121)
    Python  3.11.9 (you have 3.11.4)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
`low_cpu_mem_usage` was None, now set to True since model is quantized.
/home/jeromeku/.virtualenvs/work-env/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
./llama-10m does not have a padding token! Will use pad_token = <unk>.


In [2]:
bs, seqlen, in_features = 1, 16, 4096
dtype = torch.bfloat16
hidden_dim = model.config.hidden_size

input_ids = torch.randint(0, model.config.vocab_size, (bs, seqlen), device="cuda")
hidden_states = torch.randn(
    bs, seqlen, hidden_dim, dtype=dtype, device="cuda", requires_grad=True
)
labels = input_ids.detach().clone()
attention_mask = torch.ones((bs, seqlen), device="cuda")

# ref_out = model(input_ids, labels=labels, attention_mask=attention_mask)
# ref_head = model.lm_head

In [4]:
embed_out = model.model.embed_tokens(input_ids)
decoder_out = model.model.

In [8]:
llama_out = model.model(input_ids, attention_mask=attention_mask)

In [14]:
llama_out["last_hidden_state"].requires_grad

True

In [15]:
# Need to set inputs require_grad (i.e., output of embeddings requires grad for fused_cel to work)
model = prepare_model_for_kbit_training(
    model, use_gradient_checkpointing=training_args.gradient_checkpointing
)

In [16]:
patched_model = patch_model_fused_cel(model, use_fused_cel=True)

trainer = SFTTrainer(
    model=patched_model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,  # Can make training 5x faster for short sequences.
    args=training_args,
)
trainer.remove_callback(ProgressCallback)
_ = trainer.add_callback(MetricsCallBack())
train_stats = trainer.train()

max_steps is given, it will override any value given in num_train_epochs


  0%|          | 0/5 [00:00<?, ?it/s]

in_feat.requires_grad =  False


UnboundLocalError: cannot access local variable 'grad_in_feat' where it is not associated with a value